<a href="https://colab.research.google.com/github/AdrianTT723/Reto-Cantidad-Producida/blob/main/Reto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. IMPORTAR LIBRERÍAS Y MONTAR GOOGLE DRIVE
import pandas as pd
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import openpyxl
from openpyxl.drawing.image import Image
import os

# Montar Google Drive para acceder a los archivos
drive.mount('/content/drive')

# 2. LIMPIEZA Y PREPARACIÓN DE DATOS
# Rutas de los archivos
input_path = "/content/drive/MyDrive/RETO CIENCIAS DE DATOS/tabela289 (2).xlsx"
output_path_base = "/content/drive/MyDrive/RETO CIENCIAS DE DATOS/"
clean_file_path = f"{output_path_base}Base_Limpia.xlsx"

# Cargar el archivo de Excel original
df = pd.read_excel(input_path)

# Reemplazar los caracteres no deseados con 0
df.replace(["-", "...", ".."], 0, inplace=True)

# Eliminar la primera columna ('Variável') y la primera fila
df_cleaned = df.drop(columns=df.columns[0])
df_cleaned = df_cleaned.iloc[1:].reset_index(drop=True)

# Exportar la base de datos limpia
df_cleaned.to_excel(clean_file_path, index=False, header=False) # Se guarda sin header para facilitar la lectura posterior

print(f"Base de datos limpia guardada en: {clean_file_path}")

# 3. PREPARACIÓN DE DATOS PARA GRÁFICAS Y WIDGETS
# Leer el archivo limpio, que ahora no tiene la primera columna/fila basura
df_for_plotting = pd.read_excel(clean_file_path, header=None)

# Asignar la primera fila como los encabezados de las columnas
df_for_plotting.columns = df_for_plotting.iloc[0].values
# Eliminar las filas que no son datos (la primera que estaba vacía y la segunda que ahora es el header)
df_for_plotting = df_for_plotting.iloc[1:].reset_index(drop=True)

# Convertir todas las columnas de datos a tipo numérico para poder graficar
for col in df_for_plotting.columns[1:]: # Omitir la primera columna (años)
    df_for_plotting[col] = pd.to_numeric(df_for_plotting[col], errors='coerce').fillna(0)

# 4. CREACIÓN DE LA INTERFAZ INTERACTIVA (WIDGETS)
# Lista de categorías para la lista desplegable
categorias = [
    "1 - Alimentícios (Toneladas)", "1.1 - Açaí (fruto) (Toneladas)", "1.2 - Castanha-de-caju (Toneladas)",
    "1.3 - Castanha-do-pará (Toneladas)", "1.4 - Erva-mate (Toneladas)", "1.5 - Mangaba (fruto) (Toneladas)",
    "1.6 - Palmito (Toneladas)", "1.7 - Pequi (fruto) (Toneladas)", "1.8 - Pinhão (Toneladas)",
    "1.9 - Umbu (fruto) (Toneladas)", "1.10 - Outros (Toneladas)", "2 - Aromáticos, medicinais, tóxicos e corantes (Toneladas)",
    "2.1 - Ipecacuanha ou poaia (raiz) (Toneladas)", "2.2 - Jaborandi (folha) (Toneladas)", "2.3 - Urucum (semente) (Toneladas)",
    "2.4 - Outros (Toneladas)", "3 - Borrachas (Toneladas)", "3.1 - Caucho (Toneladas)",
    "3.2 - Hevea (látex coagulado) (Toneladas)", "3.3 - Hevea (látex líquido) (Toneladas)", "3.4 - Mangabeira",
    "4 - Ceras (Toneladas)", "4.1 - Carnaúba (cera) (Toneladas)", "4.2 - Carnaúba (pó) (Toneladas)",
    "4.3 - Outras (Toneladas)", "5 - Fibras (Toneladas)", "5.1 - Buriti (Toneladas)",
    "5.2 - Carnaúba (Toneladas)", "5.3 - Piaçava (Toneladas)", "5.4 - Outras (Toneladas)",
    "6 - Gomas não elásticas (Toneladas)", "6.1 - Balata (Toneladas)", "6.2 - Maçaranduba (Toneladas)",
    "6.3 - Sorva (Toneladas)", "7.1 - Carvão vegetal (Toneladas)", "7.2 - Lenha (Metros cúbicos)",
    "7.3 - Madeira em tora (Metros cúbicos)", "8 - Oleaginosos (Toneladas)", "8.1 - Babaçu (amêndoa) (Toneladas)",
    "8.2 - Copaíba (óleo) (Toneladas)", "8.3 - Cumaru (amêndoa) (Toneladas)", "8.4 - Licuri (coquilho) (Toneladas)",
    "8.5 - Oiticica (semente) (Toneladas)", "8.6 - Pequi (amêndoa) (Toneladas)", "8.7 - Tucum (amêndoa) (Toneladas)",
    "8.8 - Outros (Toneladas)", "9.1 - Pinheiro brasileiro (nó de pinho) (Metros cúbicos)",
    "9.2 - Pinheiro brasileiro (árvores abatidas) (Mil árvores)", "9.3 - Pinheiro brasileiro (madeira em tora) (Metros cúbicos)",
    "10 - Tanantes (Toneladas)", "10.1 - Angico (casca) (Toneladas)", "10.2 - Barbatimão (casca) (Toneladas)",
    "10.3 - Outros (Toneladas)"
]

# Crear los widgets
dropdown = widgets.SelectMultiple(options=categorias, description='Categorías:', style={'description_width': 'initial'}, layout={'width': 'max-content'})
filename_input = widgets.Text(value='Mi_Reporte', placeholder='Escribe el nombre del archivo', description='Nombre archivo:', disabled=False)
export_button = widgets.Button(description="Exportar a Excel")
output_area = widgets.Output() # Área para mostrar la gráfica y los mensajes

# 5. DEFINIR LAS FUNCIONES DE INTERACCIÓN
def update_plot(change):
    """Esta función se activa cuando cambia el valor del dropdown y actualiza las gráficas."""
    with output_area:
        clear_output(wait=True)
        selected_categories = dropdown.value

        if not selected_categories:
            print("Selecciona al menos una categoría para visualizar.")
            return

        print("Generando gráficos...")
        for selected_category in selected_categories:
            # Crear la figura para la gráfica
            fig, ax = plt.subplots(figsize=(12, 6))

            # Eje X (años) y Eje Y (datos de la categoría)
            x_axis_label = df_for_plotting.columns[0]
            ax.bar(df_for_plotting[x_axis_label], df_for_plotting[selected_category])

            # Estilo de la gráfica
            ax.set_title(f'Producción de: {selected_category}', fontsize=16)
            ax.set_xlabel('Año', fontsize=12)
            ax.set_ylabel(selected_category.split('(')[-1].replace(')', '').strip(), fontsize=12)
            plt.xticks(rotation=45)
            ax.grid(True, which='both', linestyle='--', linewidth=0.5)
            plt.tight_layout()

            # Guardar la gráfica como una imagen temporal
            temp_filename = f"{output_path_base}temp_plot_{selected_category.replace(' ', '_').replace('(', '').replace(')', '')}.png"
            plt.savefig(temp_filename, bbox_inches='tight')
            plt.show()
        print("Gráficos generados. Haz clic en 'Exportar a Excel' para guardar.")


def export_to_excel(b):
    """Esta función se activa con el botón de exportar."""
    with output_area:
        filename = filename_input.value.strip()
        if not filename:
            print("❌ Error: Por favor, introduce un nombre para el archivo.")
            return

        selected_categories = dropdown.value
        if not selected_categories:
            print("❌ Error: Por favor, selecciona al menos una categoría para exportar.")
            return

        export_filepath = f"{output_path_base}{filename}.xlsx"

        # Usar ExcelWriter para guardar los datos y las gráficas en el mismo archivo
        with pd.ExcelWriter(export_filepath, engine='openpyxl') as writer:
            # Guardar los datos de las categorías seleccionadas
            cols_to_export = [df_for_plotting.columns[0]] + list(selected_categories)
            export_df = df_for_plotting[cols_to_export]
            export_df.to_excel(writer, sheet_name='Datos', index=False)

            # Crear o acceder a la hoja para las gráficas
            if 'Gráficas' not in writer.book.sheetnames:
                 ws = writer.book.create_sheet(title='Gráficas')
            else:
                 ws = writer.book['Gráficas']

            # Incrustar las imágenes de las gráficas guardadas temporalmente
            row_offset = 1 # Starting row for placing images
            for selected_category in selected_categories:
                temp_filename = f"{output_path_base}temp_plot_{selected_category.replace(' ', '_').replace('(', '').replace(')', '')}.png"
                if os.path.exists(temp_filename):
                    img = Image(temp_filename)
                    # Adjust anchor based on previous image placement
                    img.anchor = f'B{row_offset}'
                    ws.add_image(img)
                    # Estimate rows taken by the image and add some buffer
                    # Assuming a standard plot size, roughly 20-25 rows might be occupied
                    row_offset += 30 # Adjust this value as needed

                    # Clean up temporary image file
                    os.remove(temp_filename)

        print(f"✅ ¡Éxito! Archivo guardado en: {export_filepath}")

# 6. VINCULAR FUNCIONES A WIDGETS Y MOSTRARLOS
dropdown.observe(update_plot, names='value')
export_button.on_click(export_to_excel)

# Mostrar todos los componentes de la interfaz
print("📊 Interfaz de Visualización y Exportación")
display(widgets.VBox([dropdown, filename_input, export_button, output_area]))

# Generar la primera gráfica al ejecutar la celda (based on initial dropdown value if any)
update_plot(None)